# Imports

The required packages to run this notebook.  
If any are missing use ``!{sys.executable} -m pip install <package>`` to install the package in the notebook environment

In [1]:
import os.path
import gc 
import pandas as pd
import csv
import sys
from ctypes import cdll, CDLL
from urllib import parse
#!{sys.executable} -m pip install beautifulsoup4
from bs4 import BeautifulSoup

# General helper functions

## Apply patch to fix pandas memory leak issue

The code bellow comes from https://github.com/pandas-dev/pandas/issues/2659#issuecomment-415177442  
It solves a memory leak issue caused by pandas using a library that does not release memory correctly causing memory to fill up over time and eventually crash if RAM is exceded.

This may not be a problem for smaller datasets

In [2]:
# monkeypatches.py

# Solving memory leak problem in pandas
# https://github.com/pandas-dev/pandas/issues/2659#issuecomment-12021083
try:
    cdll.LoadLibrary("libc.so.6")
    libc = CDLL("libc.so.6")
    libc.malloc_trim(0)
except (OSError, AttributeError):
    libc = None

__old_del = getattr(pd.DataFrame, '__del__', None)

def __new_del(self):
    if __old_del:
        __old_del(self)
    libc.malloc_trim(0)

if libc:
    print('Applying monkeypatch for pd.DataFrame.__del__', file=sys.stderr)
    pd.DataFrame.__del__ = __new_del
else:
    print('Skipping monkeypatch for pd.DataFrame.__del__: libc or malloc_trim() not found', file=sys.stderr)

Applying monkeypatch for pd.DataFrame.__del__


## Convert a full url to a domain

In [3]:
def to_domain(full_url):
    if(parse.urlparse(full_url).netloc == ""):
        full_url = "//" + full_url
    netloc = parse.urlparse(full_url).netloc
    if netloc.startswith('www.'):
        netloc = netloc[4:]
    return netloc

## Filter dataframe with regex

The **chunksize** variable determines how much memory is used. A larger chunksize allows for faster computation, but will crash when RAM is full. If the computation crashes try restarting the kernel and/or a lower chunksize. A chunksize of 1000 should be fine for most machines. On our machine memory is not an issue, but still it is something to consider when running the same code on different machines.

In [4]:
def filter_csv_content_regex(input_file, regex, chunksize, output_file=None):
    if (output_file is None):
        list_regex = [] # Appending to list before conferting to dataframe because it is computationally cheaper
        
        # Loops through the CSV in chunks and saves those that match the regex in the content column 
        for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
            # Add all rows with the regex in the content column to list_regex
            list_regex.append(chunk[chunk.content.str.contains(regex, regex= True, na=False)])
            # chunk_offset = chunk_offset + chunksize
            del chunk
            gc.collect()
        filtered_df = pd.concat(list_regex, ignore_index=True) # Convert the list to a pandas dataframe

        return filtered_df
    
    else:
        output = open(output_file, "w")
        output.close()
        pd.read_csv(input_file, header=None, nrows=1).to_csv(output_file, mode='a', header=False, index=False)

        # Loops through the CSV in chunks and saves those that match the regex in the content column 
        for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
            # Add all rows with the regex in the content column to list_regex
            chunk = chunk[chunk.content.str.contains(regex, regex= True, na=False)]
            chunk.to_csv(output_file, mode='a', header=False, index=False)
            del chunk
            gc.collect()
    

In [5]:
def filter_df_content_regex(input_df, regex, output_file=None):
    if (output_file is None):
        return input_df[input_df.content.str.contains(regex, regex= True, na=False)]
    else:
        save_dataframe(input_df[input_df.content.str.contains(regex, regex= True, na=False)], output_file)

## Filter dataframe by date

Filer csv or dataframe by date

In [6]:
def filter_csv_date(input_file, begin_date, end_date , chunksize, output_file=None):
    if (output_file is None):
        list = [] # Appending to list before conferting to dataframe because it is computationally cheaper

        # Loops through the CSV in chunks and saves those that match the given timeframe
        for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
            # Add all rows that match the given time frame
            list.append(chunk[(chunk['crawl_date'] >= begin_date) & (chunk['crawl_date'] <= end_date)])
            # chunk_offset = chunk_offset + chunksize
            del chunk
            gc.collect()
        filtered_df = pd.concat(list, ignore_index=True) # Convert the list to a pandas dataframe

        return filtered_df
    else:
        output = open(output_file, "w")
        output.close()
        pd.read_csv(input_file, header=None, nrows=1).to_csv(output_file, mode='a', header=False, index=False)

        # Loops through the CSV in chunks and saves those that match that match the given timeframe
        for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
            # Add all rows with the regex in the content column to list_regex
            chunk = chunk[(chunk['crawl_date'] >= begin_date) & (chunk['crawl_date'] <= end_date)]
            chunk.to_csv(output_file, mode='a', header=False, index=False)
            del chunk
            gc.collect()    

In [7]:
def filter_df_date(input_df, begin_date, end_date, output_file=None):
    if (output_file is None):
        return input_df[(input_df['crawl_date'] >= begin_date) & (input_df['crawl_date'] <= end_date)]
    else:
        save_dataframe(input_df[(input_df['crawl_date'] >= begin_date) & (input_df['crawl_date'] <= end_date)], output_file)

## Add an Internet archive Wayback Machine link to a dataframe

Use the ``crawl_date`` and ``url`` columns to estimate the Internet Archive link. The CSV only provides the date of the crawl, not the time, so the resulting link may be for the wrong capure if more than one capture is available on that date. 

In [8]:
def add_ia_url(df):
    IA_url = []
    for index, row in df.iterrows():
        url = "https://web.archive.org/web/" + str(row['crawl_date']) + "/" + row["url"]
        IA_url.append(url)
    df['IA_url'] = IA_url

## Display a specific comment

In [9]:
def display_comment(dataframe, index):
    print(dataframe['comments'][index])

## Display number of entries per domain

In [10]:
def hits_per_domain(df):
    return df['url'].apply(to_domain).value_counts()

## Intersection of two dataframes

Returns a dataframe with the intersection (overlap) of two dataframes

In [11]:
def dataframe_intersection(df1,df2):
    return pd.merge(df1, df2, how='inner')

## Dataframe difference

Returns a dataframe with all entries in each dataframe but not those in both

In [12]:
def dataframe_difference(df1,df2):
    return pd.concat([df1,df2]).drop_duplicates(keep=False, ignore_index=True)

## Dataframe complement

Returns the relative complement of df1 in df2. Meaning all elements in df2 that are not in df1. 
For CSV files it takes columns as a list input to reduce overall size and computstion time.

In [1]:
def csv_complement(csv1, csv2, columns, output_file, chunksize):
    output = open(output_file, "w")
    dw = csv.DictWriter(output, delimiter=',', fieldnames=columns)
    dw.writeheader()
    output.close()
    
    for chunk2 in pd.read_csv(csv2, iterator=True, usecols=columns, chunksize=round(chunksize/2)):
        for chunk1 in pd.read_csv(csv1, iterator=True, usecols=columns, chunksize=round(chunksize/2)):
            chunk2 = dataframe_complement(chunk1,chunk2)
            del chunk1
            gc.collect()
        chunk2.to_csv(output_file, mode='a', header=False, index=False)
        del chunk2
        gc.collect()

In [14]:
def dataframe_complement(df1,df2):
    return pd.concat([df2, df1, df1]).drop_duplicates(keep=False)

## Restrict domain list

The code below filters out unwanted domains from the dataset an creates a new CSV file.  
The ``domains_in_scope`` input variable is a list of the desired domains

In [15]:
def filter_csv_domains(input_file, output_file, domains_in_scope, chunksize):
    output = open(output_file, "w")
    output.close()
    pd.read_csv(input_file, header=None, nrows=1).to_csv(output_file, mode='a', header=False)
    
    # Loops through the CSV in chunks and saves those with URLs that match those in the domains_in_scope 
    for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
        chunk=chunk[chunk.url.apply(to_domain).isin(domains_in_scope)]
        chunk.to_csv(output_file, mode='a', header=False)
        del chunk
        gc.collect()


In [16]:
def filter_dataframe_domains(input_df, domains_in_scope):
    return input_df[input_df.url.apply(to_domain).isin(domains_in_scope)]

## Save dataframe

In [17]:
def save_dataframe(df, output_file):
    df.to_csv(output_file, index=False)